In [24]:
import pandas as pd
import numpy as np

In [25]:
df_ptus = pd.read_csv('ptus.csv')
df_ptus.head()
df_ptus = df_ptus[['PTU', "oids"]]
df_ptus['oids'] = df_ptus['oids'].apply(lambda x: set(x.split(',')))
df_ptus = df_ptus.explode('oids')
df_ptus.head()

,PTU,oids
0,PTU-00000,3300021420
0,PTU-00000,3300020623
0,PTU-00000,3300044999
0,PTU-00000,3300031753
0,PTU-00000,3300050346


In [26]:
df_map = pd.read_csv('df_map.csv')
df_map['Isolation Country'] = df_map['Isolation Country'].replace('Czechoslovakia', 'Czech Republic').replace('South America',np.nan).replace('Former Soviet Union',np.nan).replace('Unclassified', np.nan)
df_countries = pd.read_csv('countries_coords.csv')
df_countries['Country'] = df_countries['Country'].replace('United States', 'USA').replace('Russian Federation', 'Russia').replace('Korea, Republic of', 'South Korea').replace('Iran, Islamic Republic of', 'Iran').replace('Taiwan, Province of China', 'Taiwan').replace('Svalbard and Jan Mayen', 'Svalbard').replace('Micronesia, Federated States of', 'Micronesia')
df_countries.rename(columns={'Latitude (average)':'Latitude', 'Longitude (average)':'Longitude'}, inplace=True)
df_countries = df_countries[['Country','Latitude', 'Longitude']].set_index('Country')

In [27]:
def fix_lat(row):
    if pd.isna(row['Latitude']) and pd.notna(row['Isolation Country']):
        return df_countries.loc[row['Isolation Country'], 'Latitude']
    return row['Latitude']
    
def fix_lon(row):
    if pd.isna(row['Longitude']) and pd.notna(row['Isolation Country']):
        return df_countries.loc[row['Isolation Country'], 'Longitude']
    return row['Longitude']

df_map['Latitude'] = df_map.apply(fix_lat, axis=1)
df_map['Longitude'] = df_map.apply(fix_lon, axis=1)
df_map = df_map[['taxon_oid','Latitude','Longitude']]
df_map.head()

,taxon_oid,Latitude,Longitude
0,3300049023,38.53,-121.78
1,3300012840,43.07,-89.40
2,3300039503,63.88,-149.23
3,3300042005,41.20,-97.94
4,3300049265,38.55,-121.87


In [28]:
df_map = df_map.dropna(subset=['Latitude', 'Longitude'])

In [29]:
final_map = df_ptus.join(df_map.set_index('taxon_oid'), on='oids')
final_map.dropna(subset=['Latitude', 'Longitude']).to_csv('df_map_fixed.csv', index=False)